In [ ]:
import pandas as pd

# Lê a planilha 'Servidores.xlsx' e seleciona a aba 'Resultado da consulta'
df = pd.read_excel('Servidores.xlsx', sheet_name='Consulta')

# Filtra o DataFrame para manter apenas os registros onde a coluna 'Categoria' é igual a 'Docente'
df_docentes = df[df['Categoria'] == 'Docente']

# Converte os valores da coluna 'Regime De Trabalho' para string
# Isso garante que todos os valores estejam no mesmo formato, facilitando a filtragem e a criação do pivot
df_docentes['Regime De Trabalho'] = df_docentes['Regime De Trabalho'].astype(str)

# Define os regimes desejados que serão considerados: '20', '40' e 'DE'
regimes_desejados = ['20', '40', 'DE']

# Filtra o DataFrame para manter apenas os registros cujo 'Regime De Trabalho' está na lista de regimes desejados
df_docentes = df_docentes[df_docentes['Regime De Trabalho'].isin(regimes_desejados)]

# Cria uma tabela pivô utilizando:
# - 'Unidade De Lotação' como índice (cada linha representará uma unidade)
# - 'Regime De Trabalho' como colunas (cada coluna representará um regime)
# - 'Cpf' para realizar a contagem dos CPFs únicos (usando a função nunique)
# fill_value=0 preenche com zero quando não houver registros para algum regime naquela unidade
pivot_curso = pd.pivot_table(
    df_docentes,
    index='Unidade De Exercício',       # Agrupa os dados por Unidade De Lotação
    columns='Regime De Trabalho',     # Os diferentes regimes se transformarão em colunas
    values='Cpf',                     # Valor a ser agregado: a contagem dos CPFs
    aggfunc=pd.Series.nunique,        # Função para contar os valores únicos (CPF)
    fill_value=0                      # Preenche com 0 onde não houver dados
).reset_index()  # Reseta o índice para transformar o resultado do agrupamento em colunas normais

# Renomeia as colunas
pivot_curso = pivot_curso.rename(columns={'20': 'Regime 20', '40': 'Regime 40', 'DE': 'Regime DE'})


print(pivot_curso)

# Define o nome do arquivo
output_filename = 'Docentes_Total.xlsx'

# Exporta o DataFrame 'pivot_curso' para um arquivo Excel, sem salvar o índice como coluna
pivot_curso.to_excel(output_filename, index=False)

print(f"Arquivo gerado: {output_filename}")
